In [123]:
import gzip
import tarfile

from ipynb.fs.full.DataLoader import *
import defusedxml
from lxml import etree as ET
from bs4 import BeautifulSoup
import pandas as pd

In [73]:
def extract_perseus(path='../../data/perseus/greek-roman', file='hopper-texts-GreekRoman.tar.gz', 
                    extract_path='./sources/perseus', language='gk'):
    tar = tarfile.open(os.path.join(path, file))
    for member in tar.getmembers():
        if member.name.endswith(f'_{language}.xml'):
            tar.extract(member, path=extract_path)
    tar.close()

extract_perseus()

In [129]:
def build_glove_corpus(input_path='./sources/perseus/Classics', output_path='./sources/perseus/processed'):
    parser = ET.XMLParser(recover=True)
    files = []
    for (dirpath, _, filenames) in os.walk(input_path):
        for file in filenames:
            if file.endswith('.xml'):
                path = os.path.join(dirpath, file)
                with open(path) as fh:
                    soup = BeautifulSoup(fh, 'lxml-xml')
                try:    
                    # print(soup)
                    if author := soup.find('author'):
                        author = author.getText()
                    elif author := soup.find('docAuthor'):
                        author = author.getText()
                    else:
                        author = ''
                    
                    if title := soup.find('title', {"type": "work"}):
                        title = title.getText()
                    elif title := soup.find('title'):
                        title = title.getText()
                    
                    text = soup.find('TEI.2').find('text').find('body').getText()
                    text = text.replace('\n', ' ').upper()
                    greek = greek_converter(text)
                    with open(os.path.join(output_path, f'{author}_{title}.txt'), 'w') as fh:
                        fh.write(greek)
                        
                    
                except Exception as ex:
                    print(soup)
                    raise ex
                files.append({'dirpath': dirpath.split('/')[-2], 'file': file, 'author': author, 'title': title })
                
                # print(dirpath.split('/')[-2], file, author, title)
    return pd.DataFrame(files)

In [130]:
build_glove_corpus()

,dirpath,file,author,title
0,Arrian,arrian.acies_gk.xml,Arrian,Acies Contra Alanos
1,Arrian,arrian.periplous_gk.xml,Arrian,Periplus Ponti Euxini
2,Arrian,arrian.cuneg_gk.xml,Arrian,Cynegeticus
3,Arrian,arrian.tactica_gk.xml,Arrian,Tactica
4,Arrian,arrian.anab_gk.xml,Arrian,Anabasis
...,...,...,...,...
409,Dionysius,dh.004_gk.xml,Dionysius of Halicarnassus,De Isocrate
410,Dionysius,dh.011_gk.xml,Dionysius of Halicarnassus,De Thucydidis idiomatibus (epistula ad Ammaeum)
411,Dionysius,dh.006_gk.xml,Dionysius of Halicarnassus,De Demosthene
412,Dionysius,dh.009_gk.xml,Dionysius of Halicarnassus,De Dinarcho
